<a href="https://colab.research.google.com/github/noodlepopllc/ScriptReader/blob/main/Kokoro-tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install espeak-ng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 1 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-10

In [2]:
!pip install kokoro-onnx soundfile librosa requests ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 33.8 MB/s eta 0:00:00


In [47]:
import sys, os, requests, librosa
from kokoro_onnx import Kokoro
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Audio
import soundfile as sf
from kokoro_onnx.tokenizer import Tokenizer
import numpy as np


# uv run spacy download en_core_web_sm

class Voice(object):

    required = {'kokoro-v1.0.onnx':
                'https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/kokoro-v1.0.onnx',
                'voices-v1.0.bin':
                'https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/voices-v1.0.bin'}

    def __init__(self):
        self.check_required()
        self.kokoro = Kokoro("kokoro-v1.0.onnx", "voices-v1.0.bin")
        self.tokenizer = Tokenizer()

    def download_file_requests(self, url, filename):
        with requests.get(url, stream=True) as r:
            r.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    # If you have a large file, setting a chunk size is a good idea
                    f.write(chunk)
        print(f"Downloaded '{filename}' successfully.")

    def check_required(self):
        for k in Voice.required:
            if not os.path.exists(k):
                self.download_file_requests(Voice.required[k],k)

    def gui(self):
        #app = self.create_app()
        #app.launch()
        self.create_widgets()


    def create(self, text: str, voice: str, blend_voice_name: str = None, blend_voice_slider: float = 0.0, speed: float = 1.0):
        phonemes = self.tokenizer.phonemize(text, lang="en-us")

        # Blending
        if blend_voice_name:
            first_voice = self.kokoro.get_voice_style(voice)
            second_voice = self.kokoro.get_voice_style(blend_voice_name)
            voice = np.add(first_voice * ((blend_voice_slider) / 100), second_voice * ((100 - blend_voice_slider) / 100))
        samples, sample_rate = self.kokoro.create(
            phonemes, voice=voice, speed=speed, is_phonemes=True
        )
        return (sample_rate, samples)


    def create_widgets(self):
      inputText = widgets.Textarea(
      value='Kokoro TTS. Turning words into emotion, one voice at a time!',
      placeholder='Type something',
      description='Input Text:',
      disabled=False
      )
      voice_input = widgets.Dropdown(
        options=sorted(self.kokoro.get_voices()),
        value='af_sky',
        description='Voice:',
        disabled=False,
       )
      blend_voice_input = widgets.Dropdown(
        options=sorted(self.kokoro.get_voices()) + [None],
        value=None,
        description='Blend Voice (Optional):',
        disabled=False,
       )
      blend_voice_slider = widgets.FloatSlider(
          value=100.0,
          min=0,
          max=100.0,
          step=1.0,
          description='Blend Voice Amount:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.1f',
      )
      speed = widgets.FloatSlider(
          value=1.0,
          min=0,
          max=1.0,
          step=0.1,
          description='Speed:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.1f',
      )
      button = widgets.Button(
        description='Create',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Create audio',
        icon='check' # (FontAwesome names without the `fa-` prefix)
      )
      output = widgets.Output()
      def on_button_clicked(b):
        with output:
            output.clear_output()
            sample_rate, samples = self.create(inputText.value, voice_input.value, blend_voice_input.value, blend_voice_slider.value, speed.value)
            audio = Audio(data=samples, rate = sample_rate, autoplay=False)
            display(audio)

      button.on_click(on_button_clicked)

      display(inputText)
      display(voice_input)
      display(blend_voice_input)
      display(blend_voice_slider)
      display(speed)
      display(button, output)



In [48]:
    voice = Voice()
    voice.gui()

Textarea(value='Kokoro TTS. Turning words into emotion, one voice at a time!', description='Input Text:', plac…

Dropdown(description='Voice:', index=10, options=('af_alloy', 'af_aoede', 'af_bella', 'af_heart', 'af_jessica'…

Dropdown(description='Blend Voice (Optional):', options=('af_alloy', 'af_aoede', 'af_bella', 'af_heart', 'af_j…

FloatSlider(value=100.0, continuous_update=False, description='Blend Voice Amount:', readout_format='.1f', ste…

FloatSlider(value=1.0, continuous_update=False, description='Speed:', max=1.0, readout_format='.1f')

Button(description='Create', icon='check', style=ButtonStyle(), tooltip='Create audio')

Output()